<a href="https://colab.research.google.com/github/DOMINION-JOHN1/voting_eligibility_model/blob/main/preprocessing_of_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
# Define the file path of the np.array document
file_path = '/content/drive/My Drive/.npy'

# Load the np.array document using np.load()
loaded_array = np.load(file_path)

In [4]:
import tensorflow as tf

x=loaded_array


In [5]:
eligibility=pd.read_csv('/content/drive/My Drive/eligibility.csv')

In [6]:

eligibility['eligibility']=eligibility['eligibility'].replace('55', 'Eligible')
eligibility['eligibility']=eligibility['eligibility'].replace('35', 'Eligible')
eligibility['eligibility'] = eligibility['eligibility'].replace({'Eligible': 1, 'Not Eligible': 0})
target_label = np.array(eligibility['eligibility'])
target_label.astype('float32')
target_label.shape
y=target_label

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Print the sizes of the train and test sets
print("Training data size:", len(X_train))
print("Testing data size:", len(X_test))

Training data size: 10850
Testing data size: 2713


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

# Convolutional Layer 2
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

# Convolutional Layer 3
model.add(Conv2D(128, kernel_size=(3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(128, kernel_size=(3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flatten the feature map
model.add(Flatten())

# Fully connected layers
model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.0001), metrics=["accuracy"])


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 126, 126, 32)      128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 126, 126, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 61, 61, 64)        2

In [9]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

# Define the file path for saving the best model
checkpoint_filepath = '/content/drive/My Drive/best_model.h5'

# Create callbacks
checkpointer = ModelCheckpoint(filepath=checkpoint_filepath, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=5, min_lr=1e-6)

epochs = 100
batch_size = 15


# Fit the model with callbacks
hist = model.fit(X_train, y_train, batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=[checkpointer, early_stopping, reduce_lr],
    verbose=2
)


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.57750, saving model to /content/drive/My Drive/best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


724/724 - 36s - loss: 0.7298 - accuracy: 0.6553 - val_loss: 0.5775 - val_accuracy: 0.7357 - lr: 0.0010 - 36s/epoch - 50ms/step
Epoch 2/100

Epoch 2: val_loss did not improve from 0.57750
724/724 - 17s - loss: 0.6089 - accuracy: 0.7118 - val_loss: 0.5969 - val_accuracy: 0.7346 - lr: 0.0010 - 17s/epoch - 24ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.57750
724/724 - 15s - loss: 0.5874 - accuracy: 0.7312 - val_loss: 0.5789 - val_accuracy: 0.7346 - lr: 0.0010 - 15s/epoch - 21ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.57750 to 0.56853, saving model to /content/drive/My Drive/best_model.h5
724/724 - 18s - loss: 0.5768 - accuracy: 0.7319 - val_loss: 0.5685 - val_accuracy: 0.7350 - lr: 0.0010 - 18s/epoch - 25ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.56853 to 0.56391, saving model to /content/drive/My Drive/best_model.h5
724/724 - 17s - loss: 0.5786 - accuracy: 0.7335 - val_loss: 0.5639 - val_accuracy: 0.7342 - lr: 0.0010 - 17s/epoch - 23ms/step
Epoch 6

In [10]:
#